From https://pytorch.org/docs/0.3.1/nn.html?highlight=crossentropy#bcewithlogitsloss

This loss combines a Sigmoid layer and the BCELoss in one single class. This version is more numerically stable than using a plain Sigmoid followed by a BCELoss as, by combining the operations into one layer, we take advantage of the log-sum-exp trick for numerical stability.

The loss can be described as:
ℓ(x,y)=L={l1,…,lN}⊤,ln=−wn[tn⋅logσ(xn)+(1−tn)⋅log(1−σ(xn))],

where N

is the batch size. If reduce is True, then
ℓ(x,y)={mean(L),sum(L),ifsize_average=True,ifsize_average=False.

This is used for measuring the error of a reconstruction in for example an auto-encoder. Note that the targets t[i] should be numbers between 0 and 1.
Parameters:	

    weight (Tensor, optional) – a manual rescaling weight given to the loss of each batch element. If given, has to be a Tensor of size “nbatch”.
    size_average – By default, the losses are averaged over observations for each minibatch. However, if the field size_average is set to False, the losses are instead summed for each minibatch. Default: True



## Réseau de neurones

### Lancer l'apprentissage ou charger les données

In [1]:
!ls -l {path}
#!rm {path}

ls: {path}: No such file or directory


In [2]:
%%writefile scan.py
from Where import *

path = '2018-05-23_classification_BCELoss.pt'



for lr_ in lr*np.logspace(-1, 1, 9, base=10):
    path_ = path + str(lr_)
    if os.path.isfile(path_):
        net.load_state_dict(torch.load(path_))
        #print('Loading file', path)
    else:
        net = Net(n_feature=N_theta*N_orient*N_scale*N_phase, n_hidden1=n_hidden1, n_RNN=n_RNN, n_hidden2=n_hidden2, n_output=N_orient*N_scale)
        optimizer = torch.optim.SGD(net.parameters(), lr=lr_)
        loss_func = torch.nn.BCEWithLogitsLoss()

        print('Training model... with lr=', lr_)
        N_epochs = 2
        for epoch in range(N_epochs):          #max number of training epochs
            net = train(net, sample_size, optimizer=optimizer, verbose=0)                 #starting the learning
            torch.save(net.state_dict(), path_) #save the neural network state
            
    print('Training model... with lr=', lr_, 'Final loss= ', evaluate(net, sample_size, optimizer=optimizer))                 #starting the learning


Overwriting scan.py


In [3]:
%run scan.py

Loading accuracy...
[[0.0974 0.0974 0.0974 ... 0.0974 0.0974 0.0974]
 [0.0974 0.0974 0.0974 ... 0.0974 0.0974 0.0974]
 [0.0974 0.0974 0.0974 ... 0.0974 0.0974 0.0974]
 ...
 [0.0974 0.0974 0.0974 ... 0.0974 0.0974 0.0974]
 [0.0974 0.0974 0.0974 ... 0.0974 0.0974 0.0974]
 [0.0974 0.0974 0.0974 ... 0.0974 0.0974 0.0974]]


NameError: name 'evaluate' is not defined